In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.linear_model import LassoCV, Lasso

from math import sqrt
from sklearn import pipeline,ensemble
np.set_printoptions(suppress=True, precision=4)
plt.rcParams['figure.figsize'] = 10, 6
%matplotlib inline

In [ ]:
df = pd.read_csv("C:/ML Data Sets/training_data.csv")
test = pd.read_csv("C:/ML Data Sets/test_data.csv")

In [ ]:
df.head()

In [ ]:
test.head()

# Data Dictionary

In [ ]:
df.info()

In [ ]:
# Check for null values
df_null_idx = df[df.isnull().sum(axis = 1) > 0].index
df.iloc[df_null_idx]

# no null values in the data set

In [ ]:
# checking the distribution of shares and finding any outliers

plt.figure(figsize = (8,6))
plt.subplot(2,1,1)
df.shares.plot.hist(bins = 5,density = True)
plt.subplot(2,1,2)
df.shares.plot.box(vert = False)

# no outliers

In [ ]:
import hashlib
def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return bytearray(hash(np.int64(identifier)).digest())[-1] < 256 * test_ratio
    
    
def train_test_split(df,test_size,id_column,hash = hashlib.md5):
    ids = df[id_column]
    in_test_set = ids.apply(lambda id_:test_set_check(id_,test_size,hash))
    return df.loc[~in_test_set],df.loc[in_test_set]

In [ ]:
train,validation = train_test_split(df,0.3,"url_id")
print(train.shape)
print(validation.shape)

In [ ]:
# Seperating the data set into dependent and independent parts 
X_train = train.iloc[:,:-1]
X_test = validation.iloc[:,:-1]
y_train = train["shares"]
y_test = validation["shares"]
print("X_train: ",X_train.shape)
print("y_train:",y_train.shape)
print("X_test:",X_test.shape)
print("y_test:",y_test.shape)

In [ ]:
pipe = pipeline.Pipeline([
    ("poly",PolynomialFeatures(degree = 2)),
    ("scaler", StandardScaler()),
    ("fit",ensemble.GradientBoostingRegressor(random_state=1))
])
param_grid = {
    "fit__learning_rate": [0.1, 0.01],
    "fit__alpha": np.linspace(0.001, 0.999, 5),
}
gs = model_selection.GridSearchCV(cv=5,estimator = pipe,param_grid = param_grid,verbose = True,scoring = "r2",n_jobs = -1)
gs.fit(X_train,y_train)
print("best_params",gs.best_params_,"best_scores",-gs.best_score_)